In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
'''
Created on 6/12c/2018
label doc from Boson NLP
@author: WD
'''

# import time
import re
import string
import sys
from dateutil.parser import parse
# from multiprocessing import Pool
import uuid
from datetime import *
import os
import pandas as pd
import numpy as np
import html2text
import codecs




In [ ]:
def extend_amount_money(x):
    extensions = []
    x = float(x)

    #form 1: original, 181000006.3
    extensions.append('{:.1f}'.format(x))

    #form 2: 181000006.30
    extensions.append('{:.2f}'.format(x))

    #form 3: 181,000,006.30
    extensions.append('{:,.2f}'.format(x))

    # form 4: 181,000,006.3
    extensions.append('{:,.1f}'.format(x))

    print('\n'.join(extensions))

    return extensions

def extend_amount_share(x):
    extensions = []
    x = float(x)

    #form 1: original, 181000006
    extensions.append('{:.0f}'.format(x))

    # form 4: 181,000,006
    extensions.append('{:,.0f}'.format(x))

    # x = x/10000
    # # form 1: original, 181000006
    # extensions.append('{:.0f}'.format(x) + '万')
    #
    # # form 4: 181,000,006
    # extensions.append('{:,.0f}'.format(x) + '万')

    print('\n'.join(extensions))

    return extensions



def clean_dingzeng_labels(df):
    df['lockup'] = df['lockup'].fillna(0).astype(int)
    return df

In [ ]:
# def locate_labels(labels, text):
    print(labels)
    indices_target = [m.start() for m in re.finditer(labels['target'], text)]
    indices_issue_mode = [m.start() for m in re.finditer(labels['issue_mode'], text)]

    if pd.isna(labels['share']) or labels['share'] in (None, 'NaN', np.nan):
        indices_share = None
    else:
        share = str(labels['share'])
        indices_share = [m.start() for m in re.finditer(share, text)]

    if pd.isna(labels['money']) or labels['money'] in (None, 'NaN', np.nan):
        indices_money = None
    else:
        money = str(int(labels['money']))
        indices_money = [m.start() for m in re.finditer(money, text)]
    indices_lockup = [m.start() for m in re.finditer(str(labels['lockup']), text)]
    indices_buy_mode = [m.start() for m in re.finditer(labels['buy_mode'], text)]

    print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
    print(labels['target'], indices_target)
    # print([text[idx-20, idx+20] for idx in indices_target])
    print('\n||||||||||||||||||||||||||||||||||\n'.join([text[idx-100:idx+100] for idx in indices_target]))
    print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
    print(labels['issue_mode'], indices_issue_mode)
    print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
    print(labels['share'], indices_share)
    print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
    print(labels['money'], indices_money)
    print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
    print(labels['lockup'], indices_lockup)
    print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
    print(labels['buy_mode'], indices_buy_mode)
    print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
    print(labels['money'], indices_money)


In [3]:
if __name__ == '__main__':
    print('wft')
    path_proj_foldre = "C:\\projects\\fddc\\"
#     path_proj_foldre = "F:\\fddc\\"
    path_dir_pdf_dz_train = path_proj_foldre + "data_official\\round1_train_20180518\\dingzeng\\pdf"
    path_dir_html_dz_train = path_proj_foldre + "data_official\\round1_train_20180518\\dingzeng\\html"
    path_label_dz_train = path_proj_foldre + "data_official\\round1_train_20180518\\dingzeng\\dingzeng.train"
    print('++++++++++++++++++++++++++++++++++++++++ start running ++++++++++++++++++++++++++++++++++++++++++++++++++')
    s = open(path_label_dz_train, mode='r', encoding='utf-8-sig').readlines()

    my_cols = ["id_doc", "target", "issue_mode", "share", "money", "lockup", "buy_mode"]
    df_label_dz = pd.read_csv(path_label_dz_train,
                              names = my_cols,
                              sep='\t',
                              # encoding = 'utf-8-sig',
                              # header=None,
                              # engine='python',
                              # error_bad_lines=False
                              )
    df_label_dz = clean_dingzeng_labels(df_label_dz)
    # print(df_label_dz.head(5))
    print('Training label size', df_label_dz.shape)

    # scan each trainin label
    for root, dirs, files in os.walk(path_dir_html_dz_train):
        for file in files:
            #find corresponding labels
            doc_id = file.split('.')[0]
            df_labels_cur = df_label_dz[df_label_dz['id_doc'] == int(doc_id)]
            if df_labels_cur.shape[0] == 0:
                continue

            #load file text
            path_file = os.path.join(root, file)
            handle_file = codecs.open(path_file, mode='r', encoding='utf8')
            txt_html = handle_file.read()
            txt = html2text.html2text(txt_html)
            # print(txt)

            #match labels to txt
            for idx, labels in df_labels_cur.iterrows():
                target = labels['target']
                issue_mode = labels['issue_mode']
                share = labels['share']
                money = labels['money']
                lockup = labels['lockup']
                buy_mode = labels['buy_mode']
                locate_labels(labels, txt)
            break

wft
++++++++++++++++++++++++++++++++++++++++ start running ++++++++++++++++++++++++++++++++++++++++++++++++++
Training label size (13670, 7)
id_doc        100104
target          富邦人寿
issue_mode        定价
share          5e+07
money            NaN
lockup            36
buy_mode          现金
Name: 13306, dtype: object
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
富邦人寿 [32583, 32598, 32675, 33914, 34001, 34264, 34398, 34559, 34623, 34772, 34846, 35697, 36234, 36855]
 年、2013 年、2014 年 1-9 月  
本次发行/本次非公开发行/本次非公开发行股票  |  指  |  辽宁成大股份有限公司本次非公开发行不超过 10,000 万股 A 股股票的行为  
富邦人寿  |  指  |  富邦人寿保险股份有限公司  
前海基金  |  指  |  前海开源基金管理有限公司  
  
《认购协议》  |  
指  |  辽宁成大股份有限公司与富邦人寿保险股份
||||||||||||||||||||||||||||||||||
年 1-9 月  
本次发行/本次非公开发行/本次非公开发行股票  |  指  |  辽宁成大股份有限公司本次非公开发行不超过 10,000 万股 A 股股票的行为  
富邦人寿  |  指  |  富邦人寿保险股份有限公司  
前海基金  |  指  |  前海开源基金管理有限公司  
  
《认购协议》  |  
指  |  辽宁成大股份有限公司与富邦人寿保险股份有限公司签署的附条件生效的《关
||||||||||||||||||||||||||||||||||
股票的行为  
富邦人寿  |  指  |  富邦人寿保险股份有限公司  
前海基金  |  指 